# Hazus Ruleset Inference Example

Written by yisangriB, fmk

Dec 3, 2024

The example shows how the **StructuralType** and **ReplacementCost** can be inferred from basic building attributes such as **OccupancyClass, Geolocation, NumberOfStories**, and **PlanArea**, using the rulesets in [Hazus Inventory Technical Manual 6](https://www.fema.gov/sites/default/files/documents/fema_hazus-6-inventory-technical-manual.pdf) (Hazus 6) . Below image illustrates the flow of information (dependency) between building features along with the infomration sources supported in Brails ++. Note that both **StructuralType** and **ReplacementCost** are also available dierectly from NSI inventory, and users can freely choose between NSI-provided and Hazus-infered values as they see either of those better fit. The comparison of two alternatives are provided at the end of this example.



<figure style="text-align:center; margin-bottom:20px;">
    <img src="dependency.png" width="85%"/>
    <figcaption>Dependency between different building attributes</figcaption>
</figure>




Two types of hazus-based inferers are provided. "HazusInferer" adopts the original rulesets provided in [Hazus 6](https://www.fema.gov/sites/default/files/documents/fema_hazus-6-inventory-technical-manual.pdf) while "SimCenterInferer" additionally use the below three commonly applied rulesets by default. Users can also turn off some of these options if needed:
* RES3 buildings with 4 units or less (RES3A or RES3B) could be considered as RES1 because their structure types are typically much closer to single family than large multi family
* URM buildings could be turned off because in some cities these have been eliminated through mandatory retrofits.
* MH buildings could be turned off because some cities just don't have mobile homes

FEMA has recently released Hazus Inventory Technical Manual 6.1 (Hazus 6.1). The updated rulesets will be incorporated in the future release. 

In [ ]:
# Written: sy Dec 2024
# License: BSD-2

"""
 Purpose: Testing Inferer
"""

# Install packages. They are used only for the map visualization 
try:
    import folium
except Exception as e:
    import sys
    !{sys.executable} -m pip install folium
    import folium

try:
    import plotly.express as px
except Exception as e:
    import sys
    !{sys.executable} -m pip install plotly
    import plotly.express as px

import os
import sys
import copy
import json
import gc # to collect memory

import numpy as np

sys.path.insert(0, "../../")
from brails.utils import Importer
from brails.types.image_set import ImageSet    
from brails.types.asset_inventory import Asset, AssetInventory


## Step1: Set the regional boundary by its name

Target city is Hayward, CA

In [ ]:
region_data = {"type": "locationName", "data": "Hayward"}

In [ ]:
importer = Importer()
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

In [ ]:
#visualize the regional bound
bound = region_boundary_object.get_boundary()
bound[0]

## Step2: Obtain baseline inventory data by marging OSM foot print with NSI database

In [ ]:
# Scrape NSI database
nsi_class = importer.get_class('NSI_Parser')
nsi = nsi_class()
nsi_inventory = nsi.get_raw_data(region_boundary_object)

print('Total number of assets detected using NSI is '
      f'{len(nsi_inventory.inventory)}')

In [ ]:
# Scrape the building inventory from OSM:
scraper_class = importer.get_class('OSM_FootprintScraper')
scraper = scraper_class({'length': 'ft'})
scraper_inventory = scraper.get_footprints(region_boundary_object)

In [ ]:
# Merge these footprints with the NSI raw data previously downlaoded:
# Remeber to set "get_extended_features" to be true to get splitlevel and basement information
new_inventory = nsi.get_filtered_data_given_inventory(scraper_inventory, "ft", get_extended_features=True)

In [ ]:
print("Asset id=6, After Marging NSI and OSM")
new_inventory.get_asset_features(6)[1]  # empty or 'NA' are missing values

## Step 3. Run imputation to fill in missing values

Imputer fills in missing values when a feature is available for a subset of buildings in a region but not for all buildings, and it imputes the values by assuming a similarity between neighboring buildings. Therefore, the assumption is that the feature is available at least for a subset of buildings, and those buildings provide reliable information of the neighboring buildings.

If a feature is completely missing for the entire building stock in the area, that feature cannot be 'imputed.' Such features may be 'inferred' using external rulesets (Hazus or user-provided). See Step 4 for more details.

### Importing imputer and run imputation

In [ ]:
importer = Importer()
knn_imputer_class = importer.get_class("KnnImputer")

imputer=knn_imputer_class(new_inventory,n_possible_worlds=1,exclude_features=['lat','lon','fd_id'])
new_inventory = imputer.impute()

In [ ]:
print("Asset id=6, After imputation")
new_inventory.get_asset_features(6)[1]  # empty or 'NA' are missing values

## Step 4. Collect Income information (Let's randomly generate it for now)

At this point, Brails ++ does not automatically scrape the household income information. The user can import it from a CSV file or use a user-defined inferer to augment the information.
For now, let's just randomly generate it from a lognormal distribution. The income information source will be integrated into brails++ in the future. 

In [ ]:
# Temporary add incomes
import numpy as np

cal_avg = 78672 # state average
std_dev = cal_avg*0.5 # 50% cov

mean_original = cal_avg     # Mean in the original space (lognormal)
std_dev_original = std_dev     # Standard deviation in the original space (lognormal)
sample_size = 1         # Number of samples to generate
# Step 1: Calculate the parameters of the underlying normal distribution
mu = np.log(mean_original**2 / np.sqrt(std_dev_original**2 + mean_original**2))
sigma = np.sqrt(np.log(1 + (std_dev_original**2 / mean_original**2)))

# Step 2: Generate the lognormal sample using the parameters of the normal distribution
for key, val in new_inventory.inventory.items():
    lognormal_sample = np.random.lognormal(mean=mu, sigma=sigma, size=sample_size)
    val.add_features({"Income":lognormal_sample[0]})


In [ ]:
print("Asset id=6, After adding income")
new_inventory.get_asset_features(6)[1]  # empty or 'NA' are missing values

## Step 5. Run inference algorithm

Currently, we support only the **StrcturalType** and **ReplacementCost** to be inferred from the correlated building features. While NSI already provide the values of those features, let's assume those values are not available and infer them using the Hazus rulesets. This is particularly useful when user have a more reliable, high-resolution local/private data sources (e.g. for year built, income, number of stories etc.) and wants to re-evaluate the **StructuralTypes** and **ReplacementCost** and based on the user-provided inventory.


In [ ]:
# The names of existing keys to be used "predictors"
year_key = 'erabuilt' # strtype
occ_key = 'occupancy' # strtype, repcost
income_key = 'Income' # repcost
nstory_key = 'numstories' # repcost
planarea_key = 'fpAreas' # repcost
split_key = 'splitlevel' # repcost

# THe names of new keys to be used inferred.
strtype_key = 'StructureTypeHazus'
repcost_key = 'ReplacementCostHazus'

# number of possible worlds (number of realizations)
n_pw = 10

Note that the inferred values will be saved under the new keys 'StructureTypeHazus' and 'ReplacementCostHazus'. Meanwhile, NSI-provided estimates are available under 'constype' and 'repaircost'. 


Let's try both "HazusInferer" and "SimCenterInferer", just to compare the results

In [ ]:

simcenter_inferer_class = importer.get_class("SimCenterInferer")
inferer=simcenter_inferer_class(input_inventory=new_inventory, 
                                        n_possible_worlds=n_pw, 
                                        yearBuilt_key = year_key, 
                                        occupancyClass_key = occ_key, 
                                        numberOfStories_key = nstory_key, 
                                        income_key=income_key, 
                                        planArea_key=planarea_key, 
                                        splitLevel_key=split_key, 
                                        structureType_key = strtype_key, 
                                        replacementCost_key=repcost_key)
new_inventory_simcenter = inferer.infer()

In [ ]:
hazus_inferer_class = importer.get_class("HazusInferer")
inferer=hazus_inferer_class(input_inventory=new_inventory, 
                                        n_possible_worlds=n_pw, 
                                        yearBuilt_key = year_key, 
                                        occupancyClass_key = occ_key, 
                                        numberOfStories_key = nstory_key, 
                                        income_key=income_key, 
                                        planArea_key=planarea_key, 
                                        splitLevel_key=split_key, 
                                        structureType_key = strtype_key, 
                                        replacementCost_key=repcost_key)
new_inventory_hazus = inferer.infer()


In [ ]:
print("Asset id=6, After inference (SimCenterInferer)")
new_inventory_simcenter.get_asset_features(16)[1]  # empty or 'NA' are missing values

In [ ]:
_ = new_inventory_hazus.write_to_geojson('Berkeley_building.geojson')

## Step 6: Visualize results
The original inventory is too big. Let's select a random subset of buildings with size of 2395 to save some memory space for visualization

In [ ]:
inventory_simcenter_subset = new_inventory_simcenter.get_random_sample(nsamples=2395, seed=42)
inventory_hazus_subset = new_inventory_hazus.get_random_sample(nsamples=2395, seed=42)

In [ ]:
# color rules - for simplicity let's group the structural type by material
category_colors = {
    'W1': 'blue',
    'W2': 'blue',
    'W': 'blue',
    'RM1': 'green',
    'RM2': 'green',
    'M': 'green',
    'C1': 'orange',
    'C2': 'orange',
    'C3': 'orange',
    'C': 'orange',
    'S1': 'black',
    'S2': 'black',
    'S3': 'black',
    'S4': 'black',
    'S5': 'black',
    'S': 'black',
    'PC1': 'pink',
    'PC2': 'pink',
    'MH': 'red',
    'URM': 'grey',
    'H1': 'grey',
    'H': 'grey',
    'nan': 'yellow',
}

Below are some handy visualization functions to draw polygon/scatter maps from brails inventory

In [ ]:

def geo_polygon(new_inventory_simcenter, feature_key, feature_type, pw_id=0, str_colors=None, min_value=None, max_value=None, title=""):

    print(title)
    new_inventory_re1=new_inventory_simcenter.get_world_realization(pw_id)

    inventory_footprints = new_inventory_re1.get_coordinates()
    geojson_data = new_inventory_re1.get_geojson()
    # print(list(geojson_data['features'][0]['properties']))

    feature_list = list(geojson_data['features'][0]['properties'])

    center = (37.64, -122.09)
    if feature_type=="float":
        log_colormap = folium.LinearColormap(['green', 'yellow', 'red'], vmin=np.log(min_value), vmax=np.log(max_value))
        def style_function(feature):
            feature_val = feature['properties'].get(feature_key, min_value)  # Default to min_value if not found
            log_feature_val = np.log(feature_val)  # Apply the logarithmic transformation
            color = log_colormap(log_feature_val)  # Get the color from the log scale colormap
            return {'fillColor': color, 'color': color, 'weight': 2, 'fillOpacity': 0.5}
            
    elif feature_type=="str":
        def style_function(feature):
            # Get the category from the feature properties (assuming the category is stored under the 'category' key)
            category = feature['properties'].get(feature_key, 'nan')  # Default to 'nan' if no category is found
            color = str_colors.get(category, 'yellow')
            return {'fillColor': color, 'color':color , 'weight': 2, 'fillOpacity': 0.5}

    m = folium.Map(location=center, tiles="cartodbpositron", zoom_start=12)
    #m = folium.Map(zoom_start=13)
    g = folium.GeoJson(
        geojson_data,
        name="geojson",
        tooltip=folium.GeoJsonTooltip(fields=feature_list, sticky=False), # features of the first asset
        style_function=style_function
    ).add_to(m)

    from IPython.display import display, IFrame, HTML

    display(HTML(f"""
        <div style="width: 600px; height: 450; border: 2px solid black; padding: 10px;">
            {m._repr_html_()}
        </div>
    """))
    gc.collect()
    return 

def geo_scatter(new_inventory_simcenter, feature_key, feature_type, pw_id=0, str_colors=None, float_log = None, min_value=None, max_value=None, title=""):
    new_inventory_re1=new_inventory_simcenter.get_world_realization(pw_id)
    inventory_new_df, geom_new_df, nbldg = new_inventory_re1.get_dataframe()
    feature_list = list(inventory_new_df.columns)
    feature_list.remove('lat')
    feature_list.remove('lon')
    if feature_type=="float":

        if float_log:
            log_values = np.log(inventory_new_df[feature_key].astype(float) + 1e-6)
        else:
            log_values = inventory_new_df[feature_key].astype(float)
            
        colorscale = [
            [0, 'green'],   # 0 (min value)
            [0.5, 'yellow'], # Midpoint (log scale)
            [1, 'red']      # 1 (max value)
        ]
        fig = px.scatter_mapbox(inventory_new_df, 
                               lat=geom_new_df["Lat"], 
                               lon=geom_new_df["Lon"], 
                               color = log_values,
                               zoom=11, 
                               mapbox_style='open-street-map',
                               hover_data=feature_list,
                               color_continuous_scale = colorscale,
                               width=600, 
                               height=450,
                               title = title)

        if float_log:
            fig.update_layout(
                coloraxis_colorbar=dict(title=f"logscale",),
                coloraxis=dict(cmin=np.log(min_value), cmax=np.log(max_value)),
            )
    
    elif feature_type=="str":
        fig = px.scatter_mapbox(inventory_new_df, 
                                lat=geom_new_df["Lat"], 
                                lon=geom_new_df["Lon"], 
                                color=inventory_new_df[feature_key].astype(str), 
                                range_color=[-1, 4], 
                                zoom=11, 
                                mapbox_style='open-street-map',
                                color_discrete_map=str_colors,
                                width=600, 
                                height=450,
                                hover_data=feature_list,
                               title = title)
    fig.show()
    gc.collect()
    return

## Comparing Hazus vs SimCenter Inference 

Note that SimCenter inferer bases on Hazus inferer, but it additionally takes into account the three heuristic rulesets

In [ ]:
geo_scatter(inventory_simcenter_subset, 'constype', 'str', str_colors = category_colors, title="SimCenter inferer")
geo_scatter(inventory_hazus_subset, 'StructureTypeHazus', 'str', str_colors = category_colors, title="Hazus inferer")

Note the slight difference between the two results.

Below is the polygon version of the same figures.

In [ ]:
geo_polygon(inventory_simcenter_subset, 'constype', 'str', str_colors = category_colors, title="SimCenter inferer")
geo_polygon(inventory_hazus_subset, 'StructureTypeHazus', 'str', str_colors = category_colors, title="hazus inferer")

Uncomment below to see the full buildings. But note that excuting the below commands may make your browser exceed the memory limit

In [ ]:
#### For scatter plots
# geo_scatter(new_inventory_simcenter, 'constypeHazus', 'str', str_colors = category_colors)
# geo_scatter(new_inventory_hazus, 'constypeHazus', 'str', str_colors = category_colors)

#### For polygon plots
# geo_polygon(new_inventory_simcenter, 'constypeHazus', 'str', str_colors = category_colors)
# geo_polygon(new_inventory_hazus, 'constypeHazus', 'str', str_colors = category_colors)

## Comparing NSI vs. SimCenter Inference 
### Replacement Cost (1) directly from NSI; (2) predicted by inferer (SimCenter) based on basic attributes

In [ ]:
geo_scatter(inventory_simcenter_subset, 'repaircost', 'float', float_log = True,  min_value=5.e4, max_value=1.e8,  title = 'NSI replacement cost')
geo_scatter(inventory_simcenter_subset, 'ReplacementCostHazus', 'float', float_log = True,  min_value=5.e4, max_value=1.e8,  title = 'Hazus-inferenced replacement cost')

NSI estimates show a higher replacement cost for the south-west area. The buildings with low repair costs (dark green) observed in Hazus often correspond to the auxiliary structures of the houses (e.g., cottages). 

In [ ]:
geo_polygon(new_inventory_simcenter, 'repaircost', 'float', min_value=5.e4, max_value=1.e8, title = 'NSI replacement cost')
geo_polygon(new_inventory_simcenter, 'ReplacementCostHazus', 'float', min_value=5.e4, max_value=1.e8, title = 'Hazus replacement cost')

### Construction type

In [ ]:
geo_scatter(inventory_simcenter_subset, 'constype', 'str', str_colors = category_colors, title = 'NSI construction type')
geo_scatter(inventory_simcenter_subset, 'StructureTypeHazus', 'str', str_colors = category_colors, title = 'Inferred construction type')

Note for NSI, only the material types (W,RM,C,S) are provided and the number "1" is artifically assigned in Brails++. The color trend observed in two images are in general similar, but Hazus inferer (below) gives more diverse structural types. Note that those types are assigned by random generation, utilizing the inferred probability mass function. 

In [ ]:
#geo_polygon(new_inventory_simcenter, 'constype', 'str', str_colors = category_colors)
#geo_polygon(new_inventory_simcenter, 'constypeHazus', 'str', str_colors = category_colors)